In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from transformers import BertTokenizer, BertModel, pipeline
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import ast
import re
from datetime import datetime

### 1. Import data

In [2]:
# Load an array with generated indices and similarity scores:
data = np.load(r'.\..\data\cos_similarity\recommendations_300_merged.npz')
indices = data["indices"]
similarity_scores = data["similarity_scores"]

In [3]:
# Load the DataFrame with user vectors:
file_path_user_vec = rf'.\..\data\text_analysis\user_vectors\user_vectors_merged.csv'
df_user_vectors = pd.read_csv(file_path_user_vec)
df_user_vectors

,user_ID,user_vec_0,user_vec_1,user_vec_2,user_vec_3,user_vec_4,user_vec_5,user_vec_6,user_vec_7,user_vec_8,...,user_vec_290,user_vec_291,user_vec_292,user_vec_293,user_vec_294,user_vec_295,user_vec_296,user_vec_297,user_vec_298,user_vec_299
0,AE222BBOVZIF42YOOPNBXL4UUMYA,0.022338,0.100595,-0.206153,0.065659,0.023273,0.011930,0.054218,0.009806,0.071820,...,-0.001122,0.004642,-0.001718,-0.007591,-0.007428,0.001374,-0.018390,-0.007487,0.000145,0.006247
1,AE222FP7YRNFCEQ2W3ZDIGMSYTLQ,0.022153,0.030388,-0.094107,-0.025539,0.093637,0.170470,-0.031770,-0.060638,-0.060701,...,0.001796,-0.002748,0.003149,0.006347,0.020818,-0.002077,0.013311,-0.006835,-0.007763,-0.002501
2,AE222X475JC6ONXMIKZDFGQ7IAUA,-0.012598,0.219775,0.142422,-0.017760,0.129527,0.076851,0.021589,-0.034965,-0.045799,...,-0.022207,-0.002645,0.041338,-0.015161,0.011363,-0.026213,0.002272,0.003468,-0.002008,0.002792
3,AE222Y4WTST6BUZ4J5Y2H6QMBITQ,-0.231625,-0.016032,-0.102149,0.066109,-0.050187,0.007027,0.020258,0.010995,-0.007235,...,-0.006156,-0.003489,-0.000958,0.016319,-0.011625,-0.000594,0.007983,-0.010001,0.012913,-0.001186
4,AE2232TEZOEWQLAFEX2NA6VBGMYQ,0.237254,-0.088060,0.097924,0.076692,0.160438,0.100078,-0.051536,-0.046282,0.132318,...,0.003213,-0.006024,-0.003414,0.001399,0.021043,0.006149,0.000152,-0.002788,0.010600,0.002627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630310,AHZZYVEU6QFMPFZ2HJUWR22SNK4A,-0.012253,0.347016,-0.028917,-0.132051,0.016790,0.109174,-0.088193,0.002015,0.039121,...,-0.004510,0.007740,0.009071,-0.001471,-0.009232,0.002643,-0.008750,0.005727,0.000474,0.001073
630311,AHZZZAK24AJ3JNBDUZJGHHWSRVAA,-0.199268,-0.009690,0.018537,0.045325,0.030314,-0.013304,0.056271,0.021373,0.043239,...,-0.009092,0.003253,-0.014467,0.000982,-0.000779,0.004438,0.001680,0.002672,0.002944,0.006786
630312,AHZZZJP24QUSB5XWW6MAXYBZZZSQ,0.027290,0.176118,-0.098011,0.141308,0.048202,-0.112999,0.012363,-0.028594,0.050939,...,-0.002248,0.009882,0.004362,0.001602,0.005781,-0.003056,0.002786,0.011582,-0.017130,0.006931
630313,AHZZZL7YQJA3RSA6PYK3WMFACYIQ,0.114299,-0.025805,-0.125545,-0.037366,-0.196658,-0.016408,0.053420,0.040763,-0.058073,...,0.004117,-0.000237,0.003289,-0.003952,-0.009725,-0.004431,-0.008974,0.007798,0.000113,-0.001615


In [4]:
# Load the DataFrame with product vectors:
file_path_product_vec = rf'.\..\data\text_analysis\product_vectors\product_vectors_merged.csv'
df_product_vectors = pd.read_csv(file_path_product_vec)
df_product_vectors

,product_ID,product_vec_0,product_vec_1,product_vec_2,product_vec_3,product_vec_4,product_vec_5,product_vec_6,product_vec_7,product_vec_8,...,product_vec_290,product_vec_291,product_vec_292,product_vec_293,product_vec_294,product_vec_295,product_vec_296,product_vec_297,product_vec_298,product_vec_299
0,0005946468,0.005288,0.247759,0.103427,-0.091621,0.005084,-0.099639,-0.107011,0.122461,0.012713,...,0.002068,0.009174,-0.019991,-0.003565,-0.003649,-0.017079,-0.013606,0.001881,0.015874,-0.002307
1,0123034892,0.109861,0.275601,0.000386,-0.047517,0.004198,-0.083188,-0.079647,-0.121367,0.129342,...,-0.007957,-0.009873,0.003017,0.015212,-0.005792,0.007390,0.004461,0.001337,0.009957,-0.012407
2,0124784577,-0.017208,0.132227,0.100658,-0.126151,0.059223,0.032017,-0.042822,0.087571,0.032354,...,-0.005163,-0.009594,-0.002201,0.013803,0.002335,0.004500,-0.009917,0.001281,-0.000278,-0.003500
3,0515059560,-0.209201,-0.058736,0.202120,0.062009,-0.090060,0.028901,-0.018587,0.080671,-0.056896,...,-0.010150,0.030672,-0.013799,0.020456,-0.005572,-0.030364,0.009011,-0.004363,-0.024073,0.008783
4,0615675026,-0.259519,-0.082304,0.141455,0.102475,-0.019962,0.111337,-0.059118,0.036263,-0.016532,...,0.015419,0.001891,-0.003339,-0.002042,-0.013208,-0.019155,0.014448,0.007363,-0.002628,-0.006653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115551,B0CBMFK1S2,-0.067334,0.154381,-0.197790,-0.023412,0.010574,-0.022570,0.074094,-0.099601,0.037623,...,0.010685,-0.005477,0.001734,0.002790,-0.004053,0.008070,-0.002784,-0.001871,0.003500,0.010731
115552,B0CBWDTY41,0.069057,0.038905,-0.166532,-0.235593,-0.038300,0.060362,-0.100929,0.040324,-0.045593,...,-0.010125,0.001486,0.003190,0.004283,0.005094,-0.000096,-0.000655,-0.001726,0.001275,-0.004093
115553,B0CBXM7WHY,-0.169268,0.007456,0.089662,-0.067994,0.013050,0.057634,-0.050431,-0.041138,0.025872,...,-0.009981,0.013271,-0.008269,-0.003295,0.014088,-0.008365,0.004828,0.003399,0.010965,-0.004015
115554,B0CCPDTRK7,0.118037,-0.196848,-0.070833,-0.118934,0.010600,0.104431,-0.061131,-0.024269,-0.122102,...,0.002205,0.009895,0.001180,-0.001901,0.007467,-0.010565,0.000210,0.000453,0.006803,-0.002013


In [5]:
# load the dataset of PCA normalized embeddings: 
file_path = rf'.\..\data\embeddings_dim_reduction\df_user_normalized_PCA_merged.csv' # adjust the filepath with the proper file 
df_user_normalized_PCA = pd.read_csv(file_path) 

In [6]:
# Upload the user DataFrame and analyse the recommendations
file_path = r'.\..\data\data_clean\merged_user_meta_df.csv'  # Update this with your file path
df_user = pd.read_csv(file_path)

### 2. Run recommendations for the specific user_id

### Compare Reviews - Similarity Research with COS Similarity

* Extract the user and product vectors.
* Compute the cosine similarity between each user and all products (already precalculated matrix for 300 closest vectors)
* Rank products for each user based on similarity scores.

Create a DataFrame that stores all items that user purchased, to remove those from the recommendations

In [7]:
user_item_df = df_user_normalized_PCA.groupby('user_ID')['product_ID'].apply(list).reset_index()
user_item_df

,user_ID,product_ID
0,AE222BBOVZIF42YOOPNBXL4UUMYA,[B013HR1A92]
1,AE222FP7YRNFCEQ2W3ZDIGMSYTLQ,[B0BTT658PQ]
2,AE222X475JC6ONXMIKZDFGQ7IAUA,[B00PBDMRES]
3,AE222Y4WTST6BUZ4J5Y2H6QMBITQ,[B00012FPSO]
4,AE2232TEZOEWQLAFEX2NA6VBGMYQ,[B07QNPXBLH]
...,...,...
630310,AHZZYVEU6QFMPFZ2HJUWR22SNK4A,[B07JJ9NFFH]
630311,AHZZZAK24AJ3JNBDUZJGHHWSRVAA,[B00KXFD75M]
630312,AHZZZJP24QUSB5XWW6MAXYBZZZSQ,[B00IG0677G]
630313,AHZZZL7YQJA3RSA6PYK3WMFACYIQ,[B091TKH1JF]


In [8]:
df_user_normalized_PCA['user_ID'].value_counts()

user_ID
AG73BVBKUOH22USSFJA5ZWL7AKXA      165
AEZP6Z2C5AVQDZAJECQYZWQRNG3Q      146
AEMP3A7IKW37CMWFXNKXWW6HGJHA_1    115
AGZUJTI7A3JFKB4FP5JOH6NVAJIQ_1     87
AFDYIK3FNPY2JFBQYUWC6GSBMIRQ_2     83
                                 ... 
AEX24LYNE5WPGISAP425SQPUKJXQ        1
AFSMCZTEUW3TI2BSPE25BD5GKXLA        1
AGKNUO4XOIPCSIKDRHO56UQDPXVQ        1
AEJQRDONU2O5LSOD5OC77XO43DFA        1
AGIYQU6RK6TBKBCMWKVPBPBMMJNA        1
Name: count, Length: 630315, dtype: int64

In [9]:
unique_combinations = df_user_normalized_PCA[['user_ID', 'product_ID']].value_counts()
unique_combinations

user_ID                       product_ID
AGWOOXMW2IXPKZOWAIWNMCXY7LBQ  B09NS1VG4L    2
AGPGHXWBOK3TRJU7WACF5AAYBBVQ  B0B18CG44R    1
AGPGHNJ32YJHVI2YKOBYQHSJS4WQ  B07L5QVV6W    1
AGPGHPOQZP7XKLM6KJF2E6JT3D5A  B07DLF8HKR    1
AGPGHQIMPLOJD3FR3ODRDJFYSJBQ  B079D87KKM    1
                                           ..
AFENC7VBDKGZQXZO437KEQWP45WA  B07Z2QTWSW    1
AFENCIKTUQKQM2YPBVDGBB4S6OZQ  B01KU867GM    1
AFENCVCHNNX3UATG4KRVIPYJLLGQ  B00YZ0XOWC    1
AFENDQSAV5XVGGXKHRSDE3DUYK4A  B00D4T40D2    1
AHZZZSOTVOVACVK2WWXL4ITEAPIA  B00R1TAN7I    1
Name: count, Length: 692330, dtype: int64

In [10]:
# Extract user vectors
user_ids = df_user_vectors['user_ID'].values
user_vectors = df_user_vectors.iloc[:, 1:].values

# Extract product vectors
product_ids = df_product_vectors['product_ID'].values
product_vectors = df_product_vectors.iloc[:, 1:].values

In [11]:
def recommend_top_n_products_by_user_id(user_id, user_ids, similarity_scores, indices, product_ids, user_item_df, top_n=5):
    """
    Recommend top N products for a given user based on precomputed top-N cosine similarity.

    Parameters:
    - user_id: User ID for whom to generate recommendations
    - user_ids: List of user IDs corresponding to rows in similarity data
    - similarity_scores: Precomputed top-N similarity scores (users x top-N products)
    - indices: Indices of the top-N products for each user
    - product_ids: List of product IDs corresponding to the product vectors
    - user_item_df: DataFrame containing user-product interactions
    - top_n: Number of top recommendations to return

    Returns:
    - List of (product_id, similarity_score) tuples
    """

    # Find the index of the user_ID
    if user_id not in user_ids:
        raise ValueError(f"User ID '{user_id}' not found in the user data.")
    
    user_index = np.where(user_ids == user_id)[0][0]  # Locate the index of user_id

    # Get top-N similarity scores and product indices for this user
    user_similarities = similarity_scores[user_index]
    user_product_indices = indices[user_index]

    # Retrieve the list of already purchased products for the user
    purchased_products = user_item_df.loc[user_item_df['user_ID'] == user_id, 'product_ID']
    if not purchased_products.empty:
        # Flatten and handle lists or non-hashable elements in purchased_products
        if purchased_products.apply(lambda x: isinstance(x, list)).any():
            purchased_products = set([item for sublist in purchased_products for item in sublist])
        else:
            purchased_products = set(purchased_products)
    else:
        purchased_products = set()

    # Filter and sort recommendations
    recommendations = []
    for i, product_index in enumerate(user_product_indices):
        product = product_ids[product_index]
        if product not in purchased_products:
            recommendations.append((product, user_similarities[i]))
        if len(recommendations) >= top_n:  # Stop when we have enough recommendations
            break
    
    return recommendations

# Example usage
user_id_input = "AGKHLEW2SOWHNMFQIJGBECAF7INQ"  # Replace with user_ID
top_n = 5

try:
    recommendations = recommend_top_n_products_by_user_id(
        user_id_input, 
        user_ids, 
        similarity_scores, 
        indices, 
        product_ids, 
        user_item_df, 
        top_n
    )

    print(f"Top-{top_n} Recommendations for User '{user_id_input}':")
    for product_id, score in recommendations:
        print(f"Product ID: {product_id}, Similarity Score: {score:.4f}")
except ValueError as e:
    print(e)


Top-5 Recommendations for User 'AGKHLEW2SOWHNMFQIJGBECAF7INQ':
Product ID: B081QVZ8CV, Similarity Score: 0.7219
Product ID: B081TH5HJJ, Similarity Score: 0.6884
Product ID: B08KNVV18L, Similarity Score: 0.6792
Product ID: B0BQWTXV2Q, Similarity Score: 0.6746
Product ID: B07PXFPVZK, Similarity Score: 0.6722


In [12]:
# Find users who ourchased more than 1 product to verify the recommendations for them:
duplicated_user_ids = df_user_normalized_PCA['user_ID'][df_user_normalized_PCA['user_ID'].duplicated()].unique()
duplicated_user_ids

array(['AGKHLEW2SOWHNMFQIJGBECAF7INQ', 'AFQLNQNQYFWQZPJQZS6V3NZU4QBQ',
       'AFSKPY37N3C43SOI5IEXEK5JSIYA', ...,
       'AFV3EYFZLLLBWIXWRZUSRJOHLNBA', 'AEVTGJFLW22HVSWOJLJCBJUN46WA',
       'AHURE3VT2MLCTARMYI7JA7KKDYAA'], dtype=object)

In [13]:
pd.set_option('display.max_colwidth', None)

Analyse users purchase:

In [14]:
single_user_check = df_user[df_user['user_ID']=='AGKHLEW2SOWHNMFQIJGBECAF7INQ']
single_user_check

,rating,review_title,text_review,user_images,product_ID,parent_ID,user_ID,timestamp,helpful_review_vote,user_purchase_verification,year,cleaned_text,parent_asin,cleaned_title
0,5,Such a lovely scent but not overpowering.,"This spray is really nice. It smells really good, goes on really fine, and does the trick. I will say it feels like you need a lot of it though to get the texture I want. I have a lot of hair, medium thickness. I am comparing to other brands with yucky chemicals so I'm gonna stick with this. Try it!",[],B00YQ6X8EO,B00YQ6X8EO,AGKHLEW2SOWHNMFQIJGBECAF7INQ,2020-05-05 14:08:48.923,0,1,2020,"this spray is really nice. it smells really good, goes on really fine, and does the trick. i will say it feels like you need a lot of it though to get the texture i want. i have a lot of hair, medium thickness. i am comparing to other brands with yucky chemicals so im gonna stick with this. try it!",B00YQ6X8EO,herbivore natural sea mist texturizing salt spray coconut 8 oz
1,4,Works great but smells a little weird.,"This product does what I need it to do, I just wish it was odorless or had a soft coconut smell. Having my head smell like an orange coffee is offputting. (granted, I did know the smell was described but I was hoping it would be light)",[],B081TJ8YS3,B081TJ8YS3,AGKHLEW2SOWHNMFQIJGBECAF7INQ,2020-05-04 18:10:55.070,1,1,2020,"this product does what i need it to do, i just wish it was odorless or had a soft coconut smell. having my head smell like an orange coffee is offputting. granted, i did know the smell was described but i was hoping it would be light",B081TJ8YS3,all natural vegan dry shampoo powder eco friendly root touch up hair powder volumizer for brown hair brunette and dark hair brun application brush two goats apothecary


Analyse recommended products:

In [15]:
product_ids_to_filter = ['B0170FP8CC', 'B0BQWTXV2Q','B08JQS9FVP','B09TQ2SDKK','B085TBXF1Z']  # List of product IDs you want to filter
filtered_df = df_user[df_user['product_ID'].isin(product_ids_to_filter)]
filtered_df.head(5)

,rating,review_title,text_review,user_images,product_ID,parent_ID,user_ID,timestamp,helpful_review_vote,user_purchase_verification,year,cleaned_text,parent_asin,cleaned_title
2398,4,Super Gentle,"My skin is really sensitive.<br /><br />I also don't generally like all-in-one products. You should see my bathroom cabinets. Trying to consolidate, but I get different rashes in different places, and no one thing works for everything.<br /><br />This is very gentle and did not make my skin or face or scalp break out or rash up at all. It's not super moisturizing, but it wasn't drying, either. The scent is very light, and the cleanser has a little minty/tingly kick. This would have worked for me in my teens or 20s when I couldn't wait to get out of the shower and on with my life.",[],B0BQWTXV2Q,B0BQWTXV2Q,AENH6LSB6BM7XLPEYUL43WBOD6JA,2023-02-06 18:45:25.633,0,0,2023,"my skin is really sensitive.i also dont generally like allinone products. you should see my bathroom cabinets. trying to consolidate, but i get different rashes in different places, and no one thing works for everything.this is very gentle and did not make my skin or face or scalp break out or rash up at all. its not super moisturizing, but it wasnt drying, either. the scent is very light, and the cleanser has a little mintytingly kick. this would have worked for me in my teens or 20s when i couldnt wait to get out of the shower and on with my life.",B0BQWTXV2Q,hanz de fuko everything wash for hair face body allinone soap shampoo for men women kids safe for sensitive skin minty rum sandalwood 8 oz bottle
3381,4,Strong scent but works great,"Really happy with this concentrate. I did not expect it to work well but am very thankful to be wrong. It thickens to the normal consistency of conditioners after it cools. It’s very moisturizing for my thick curly hair. My only issue was the scent. Very strong herbal scent. It was not really my thing but a little added essential oil was all it needed. However, in the future I’ll just order the unscented and add my own scent",[],B08JQS9FVP,B08JQS9FVP,AFPAGWUQX3ELC4PWOOATIP7EDMOA,2021-02-12 20:57:00.886,3,1,2021,"really happy with this concentrate. i did not expect it to work well but am very thankful to be wrong. it thickens to the normal consistency of conditioners after it cools. its very moisturizing for my thick curly hair. my only issue was the scent. very strong herbal scent. it was not really my thing but a little added essential oil was all it needed. however, in the future ill just order the unscented and add my own scent",B08JQS9FVP,ethique hydrating conditioner concentrate for balanced to dry hair bloom sulfatefree plasticfree vegan crueltyfree ecofriendly makes 1183 fl oz bottle of conditioner pack of 1
4327,3,Strongly scented,"The smell is really strong. Not at all subtle. It reminds me of scented talcum powders my Grandma used to use, kind of a spicy floral smell. I personally think it is too strong and would not want to go around smelling that way if I can help it. That said, it works well to decrease the appearance of oily roots and can extend your hair wash by a day for sure, if you can stand the smell. I will keep it as a backup for when I run out of the other lighter scented dry shampoos.",[],B0170FP8CC,B0170FP8CC,AE5S2ACTELDBCIGS2M5377BIAB7Q,2017-10-13 22:26:41.602,1,1,2017,"the smell is really strong. not at all subtle. it reminds me of scented talcum powders my grandma used to use, kind of a spicy floral smell. i personally think it is too strong and would not want to go around smelling that way if i can help it. that said, it works well to decrease the appearance of oily roots and can extend your hair wash by a day for sure, if you can stand the smell. i will keep it as a backup for when i run out of the other lighter scented dry shampoos.",B0170FP8CC,captain blankenship dry shampoo 24 ounce
8252,5,Great product in a convenient package with ingredients I recognize!,I admit I ordered this product because I loved t

In [16]:

# Display the cleaned_text column
filtered_df.cleaned_text

2398           my skin is really sensitive.i also dont generally like allinone products. you should see my bathroom cabinets. trying to consolidate, but i get different rashes in different places, and no one thing works for everything.this is very gentle and did not make my skin or face or scalp break out or rash up at all. its not super moisturizing, but it wasnt drying, either. the scent is very light, and the cleanser has a little mintytingly kick. this would have worked for me in my teens or 20s when i couldnt wait to get out of the shower and on with my life.
3381                                                                                                                                            really happy with this concentrate. i did not expect it to work well but am very thankful to be wrong. it thickens to the normal consistency of conditioners after it cools. its very moisturizing for my thick curly hair. my only issue was the scent. very strong herbal scent. it was not